In [109]:
import cv2
import numpy as np
from math import acos, pi, sqrt, cos, e

In [111]:
#Mean, Gaussian, Median Filter 모두 같은 마스크 크기 적용
#사각형이라고 가정, 보통은 정사각형이라 3*3, 5*5, 7*7을 기준으로 생각할 것
maskheight = int(input("Height of square : "))
maskwidth = int(input("Width of square : "))

Height of square : 3
Width of square : 3


In [112]:
#방법2 hsi to rgb 변환 사용 (Mean , Gaussian Filter만)

In [113]:
#위에서 지정한 크기의 마스크로, hsi -> RGB 변환 포함
def meanfilter(src, height, width, maskheight, maskwidth):
      
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    #meanfilter는 mask라 사이즈 지정 잘 해주기
    mean = np.zeros((maskheight, maskwidth), dtype=np.float32)
    plt = np.zeros((height, width,3), dtype=np.uint8)
    
    pixel_num = maskheight*maskwidth
    
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(maskheight):
        for j in range(maskwidth):
            mean[i][j] = 1/pixel_num
    
    
    #meanfilter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B_mean = np.zeros((height, width))
    hsi2G_mean = np.zeros((height, width))
    hsi2R_mean = np.zeros((height, width))
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G_mean[i][j] = ((I[i][j]*3. - (hsi2R_mean[i][j]+hsi2B_mean[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B_mean[i][j] = ((I[i][j]*3 - (hsi2R_mean[i][j]+hsi2G_mean[i][j])))
                    H[i][j] = H[i][j]+2*pi/3
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R_mean[i][j] = ((I[i][j]*3 - (hsi2G_mean[i][j]+hsi2B_mean[i][j])))
                    H[i][j] = H[i][j] + 4*pi/3
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = mean[m][n] * I[i+m][j+n]
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2R_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2G_mean[i][j] += ((temp_I*3. - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2G_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2B_mean[i][j] += ((temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n]+2*pi/3
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2B_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2R_mean[i][j] += ((temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n] + 4*pi/3
            
            
    plt = cv2.merge((hsi2B_mean, hsi2G_mean, hsi2R_mean))
    return plt

In [114]:
#위에서 지정한 크기의 마스크로
def gaussianfilter(src, height, width, maskheight, maskwidth):   
    
    I = np.zeros((height, width))
    S = np.zeros((height, width))
    H = np.zeros((height, width))


    #B, G, R = cv2.split(src)
    B = src[:,:,0]
    G = src[:,:,1]
    R = src[:,:,2]


    for i in range(height):
        for j in range(width):
            B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
            I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
            # prevent divide by 0, adding 0.00000001
            S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
            H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)
    
    
    
    #gausianfilter는 mask라 사이즈 지정 잘 해주기
    gaussian = np.zeros((maskheight, maskwidth), dtype=np.float32)
    
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    
    #sigma 입력받기. sigma가 커질 수록 분포가 넓어짐
    sigma = int(input("가우시안 필터의 시그마 값을 정수로 입력해주세요 : "))

    g_sum = 0
    #gaussian filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(maskheight):
        for j in range(maskwidth):
            gaussian[i][j] = (1/(2*pi*sigma*sigma)) * e ** (-((i**2) + (j**2))/(sigma*sigma))
            g_sum += gaussian[i][j]
    
    
    #gaussian filter 함수를 불러왔을 때 자동으로 hsi도 선언
    hsi2B_mean = np.zeros((height, width))
    hsi2G_mean = np.zeros((height, width))
    hsi2R_mean = np.zeros((height, width))
    
        
    for i in range(height):
        for j in range(width):
            
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                    hsi2B_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2R_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2G_mean[i][j] = ((I[i][j]*3. - (hsi2R_mean[i][j]+hsi2B_mean[i][j])))
            
                if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                    H[i][j] = H[i][j]-2*pi/3
                    hsi2R_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2G_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2B_mean[i][j] = ((I[i][j]*3 - (hsi2R_mean[i][j]+hsi2G_mean[i][j])))
                    H[i][j] = H[i][j]+2*pi/3
        
                if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                    H[i][j] = H[i][j] - 4*pi/3
                    hsi2G_mean[i][j] = ((1-S[i][j])*I[i][j])
                    hsi2B_mean[i][j] = ((1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j])
                    hsi2R_mean[i][j] = ((I[i][j]*3 - (hsi2G_mean[i][j]+hsi2B_mean[i][j])))
                    H[i][j] = H[i][j] + 4*pi/3
                
                continue
            
            temp_I = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp_I = 1/g_sum * gaussian[m][n] * I[i+m][j+n]
                        
                    if H[i+m][j+n] >= 0 and H[i+m][j+n] < 2*pi/3: #0도~120도
                        hsi2B_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2R_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2G_mean[i][j] += ((temp_I*3. - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
            
                    if H[i+m][j+n] >= 2*pi/3 and H[i+m][j+n] < 4*pi/3:
                        H[i+m][j+n] = H[i+m][j+n]-2*pi/3
                        hsi2R_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2G_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2B_mean[i][j] += ((temp_I*3 - (((1-S[i+m][j+n])*temp_I)+((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n]+2*pi/3
        
                    if H[i+m][j+n] >= 4*pi/3 and H[i+m][j+n] <= 2*pi:
                        H[i+m][j+n] = H[i+m][j+n] - 4*pi/3
                        hsi2G_mean[i][j] += ((1-S[i+m][j+n])*temp_I)
                        hsi2B_mean[i][j] += ((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)
                        hsi2R_mean[i][j] += ((temp_I*3 - (((1+(S[i+m][j+n]*cos(H[i+m][j+n]) / cos(pi/3-H[i+m][j+n])))*temp_I)+((1-S[i+m][j+n])*temp_I))))
                        H[i+m][j+n] = H[i+m][j+n] + 4*pi/3
            
    plt = cv2.merge((hsi2B_mean, hsi2G_mean, hsi2R_mean))
    return plt

In [115]:
# B,G,R
def compute_Hue(B, G, R):
    # prevent divide by 0, adding 0.00000001
    angle = 0.5*((R - G) + (R - B)) / sqrt((R-G)*(R-G) + (R-B)*(G-B) + 0.00000001)
    if B<= G:
        return acos(angle)
    elif B> G:
        return (2*pi- acos(angle))
    #return acos(angle) if B <= G else (2*pi - acos(angle))

In [116]:
src = cv2.imread('./Gaussian noise_1.jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
mean = np.zeros((height, width,3), dtype=np.uint8)
gaussian = np.zeros((height, width,3), dtype=np.uint8)

mean = meanfilter(src, height, width, maskheight, maskwidth)
gaussian = gaussianfilter(src, height, width, maskheight, maskwidth)
cv2.imshow("Mean Filter", mean)
cv2.imshow("Gaussian Filter", gaussian)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

가우시안 필터의 시그마 값을 정수로 입력해주세요 : 3
